## Mace MD Demo notebook

This should demonstrate the bare bones of the API for setting up a variety of MD jobs with hybrid MACE-classical hamiltonians.  The same functionalty is exposed through the `mace-md` command line program that is installed with the `mace-openmm-interop` package


Your vs code server instance should have an attached CUDA GPU for this to run!

In [20]:
!nvidia-smi

Wed Feb 22 17:32:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:C1:00.0 Off |                    0 |
| N/A   34C    P0    77W / 500W |   3942MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
from openmmtools.openmm_torch.hybrid_md import PureSystem, MixedSystem
import torch
import logging
from typing import Optional, Union
from mace import tools

In [11]:
tools.setup_logger(level="INFO", directory="./logs")

In [21]:
# Change inputs in this cell only
file = "./example_data/ejm_31.sdf"  # path to the system to be simulated, can be either protein ligand complex or isolated (sdf)
ml_mol = "./example_data/ejm_31.sdf"  # path to the proper topology of the small molecule to be modeleld by mace, since pdb files are not sufficiently detailed.  this should be sdf.
model_path = "/home/jhm72/rds/hpc-work/mace-openmm-interop/tests/example_data/SPICE_L1_N3_swa.model"
forcefields = [
    "amber/protein.ff14SB.xml",
    "amber/tip3p_standard.xml",
    "amber14/DNA.OL15.xml",
]
# name of the residue to be parametrised by mace, should correspond to whatever it is called in the ml_mol sdf file and system file
resname = "4GIH_lig_ejm_31"
# minimum separation between the solute and box wall
padding = 1.2
# ion concentration in the solute
ionicStrength = 0.15
# cutoff distance (nm) for the non-bonded terns of the classical forcefield
nonbondedCutoff = 1.0
# name of the neural network potential, should correpond to the implemented FFs
# in the openmm-ml package
potential = "mace"
# simulation temperature
temperature = 298.15
# precision to use for both MACE and openMM
dtype = torch.float64
# which version of the torch_nl to use - the n^2 impl is more reliable
# directory where simulation output will be written
output_dir = "./output"
# specify the type of system to create - pure (just the solute simulated in vcuum)
# hybrid (small molecule modelled by MACE, rest of the system (protein or solvent) modelled by classical forcefield), decoupled (for ABFE simulations - lambda parameter controls switching off the ligand non-bonded terms)

In [22]:
torch.set_default_dtype(dtype)

## Run pure MD of the small molecule in vacuum - no periodic boundary conditions

In [23]:
system = PureSystem(
    ml_mol=file,
    model_path=model_path,
    potential="mace",
    temperature=temperature,
    output_dir="output_pure_mace",
)

2023-02-22 17:35:02.620 INFO: Using SMFF openff_unconstrained-1.0.0.offxml
2023-02-22 17:35:02.620 INFO: Using SMFF openff_unconstrained-1.0.0.offxml
2023-02-22 17:35:03.341 INFO: Initialized topology with (32, 3) positions
2023-02-22 17:35:03.341 INFO: Initialized topology with (32, 3) positions
MACE model compiled
rmax from model is tensor(5., device='cuda:0')


In [24]:
system.run_mixed_md(
    steps=1000, interval=100, output_file="output_md_test.pdb", restart=False
)

2023-02-22 17:35:07.104 INFO: Minimising energy...
2023-02-22 17:35:07.104 INFO: Minimising energy...


[W parser.cpp:3513] Warning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (function operator())
[W graph_fuser.cpp:104] Warning: operator() profile_node %512 : int[] = prim::profile_ivalue(%510)
 does not have profile information (function operator())


#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,0.10000000000000007,-361.9719802207018,32.348076756719514,0
200,0.20000000000000015,-346.5240193866451,50.246644136886175,1.23
300,0.3000000000000002,-333.25356693151235,84.01805580189905,1.23
400,0.4000000000000003,-338.3268061168005,138.28311080966887,1.23
500,0.5000000000000003,-314.2289287570971,123.9026602636784,1.23
600,0.6000000000000004,-310.40543416888374,139.3714106837999,1.23
700,0.7000000000000005,-311.32704786015336,136.40829651483722,1.23
800,0.8000000000000006,-317.5036722985203,190.37509554128243,1.22
900,0.9000000000000007,-310.242170354545,177.26852466881516,1.22
1000,1.0000000000000007,-315.9279982430887,214.94723195234428,1.22


## Run Mixed MD with the ligand solvated in a tip3p water box

Construct a system where the bonded terms for the small molecule are replaced by MACE, solvent and long-range interactions retain the AMBER FF parameters

Small molecule in periodic water box

In [25]:
mixedSystem = MixedSystem(
    file=file,
    ml_mol=ml_mol,
    model_path=model_path,
    forcefields=forcefields,
    resname=resname,
    nnpify_type="resname",
    padding=padding,
    ionicStrength=ionicStrength,
    nonbondedCutoff=nonbondedCutoff,
    potential=potential,
    temperature=temperature,
    dtype=dtype,
    output_dir=output_dir,
)

2023-02-22 17:41:43.336 INFO: Using SMFF openff_unconstrained-1.0.0.offxml
2023-02-22 17:41:43.336 INFO: Using SMFF openff_unconstrained-1.0.0.offxml
2023-02-22 17:41:43.375 INFO: Initialized topology with (32, 3) positions
2023-02-22 17:41:43.375 INFO: Initialized topology with (32, 3) positions
2023-02-22 17:41:43.710 INFO: Requested to generate parameters for residue <Residue 0 (4GIH_lig_ejm_31) of chain 0>
2023-02-22 17:41:43.710 INFO: Requested to generate parameters for residue <Residue 0 (4GIH_lig_ejm_31) of chain 0>
2023-02-22 17:41:44.262 INFO: Generating a residue template for [H][c]1[n][c]([N]([H])[C](=[O])[C]([H])([H])[H])[c]([H])[c]([N]([H])[C](=[O])[c]2[c]([Cl])[c]([H])[c]([H])[c]([H])[c]2[Cl])[c]1[H] using openff_unconstrained-1.0.0.offxml
2023-02-22 17:41:44.262 INFO: Generating a residue template for [H][c]1[n][c]([N]([H])[C](=[O])[C]([H])([H])[H])[c]([H])[c]([N]([H])[C](=[O])[c]2[c]([Cl])[c]([H])[c]([H])[c]([H])[c]2[Cl])[c]1[H] using openff_unconstrained-1.0.0.offxml


In [27]:
# Once the mixed system is created, we can run several different types of simulation:

mixedSystem.run_mixed_md(
    steps=1000, interval=100, output_file="./output_md_mixed.pdb", restart=False
)

2023-02-22 17:43:41.290 INFO: Minimising energy...
2023-02-22 17:43:41.290 INFO: Minimising energy...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,0.10000000000000007,-23728.03439745035,32.27357944739915,0
200,0.20000000000000015,-23385.91202908763,56.5612858790912,4.93
300,0.3000000000000002,-23104.842156964965,77.30852742452024,4.94
400,0.4000000000000003,-22885.184702507002,97.56735655059161,4.95
500,0.5000000000000003,-22603.44355253651,110.6231556231202,4.94
600,0.6000000000000004,-22480.601338946217,128.7015887484071,4.95
700,0.7000000000000005,-22234.721713959458,139.27429104657767,4.76
800,0.8000000000000006,-21993.158125743634,159.36626860667124,4.77
900,0.9000000000000007,-21884.374032673528,170.2822174408578,4.73
1000,1.0000000000000007,-21606.423324585223,181.82755208408912,4.69


In [ ]:
# runs Markov chain monte carlo replica exchange to interpoolate between MM and MM/ML descriptions of the system
mixedSystem.run_repex(
    replicas=3, restart=False, steps=10, equilibration_protocol="gentle"
)